# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [26]:
import numpy as np
import pandas as pd 
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [27]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [28]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'G AKDEROABSLSNMA)E/LTNG (QLSCEA T', 'calculationPrice': 'close', 'open': 133.4, 'openTime': 1672561669014, 'openSource': 'icoilffa', 'close': 134.46, 'closeTime': 1618828238617, 'closeSource': 'oaflfcii', 'high': 137.12, 'highTime': 1689549359312, 'highSource': 'prdlei1tea ueyedm 5nc i', 'low': 135.38, 'lowTime': 1639420759702, 'lowSource': 'rXlt r  iaIieecmpeE', 'latestPrice': 134.3, 'latestSource': 'Close', 'latestTime': 'January 8, 2021', 'latestUpdate': 1672159751860, 'latestVolume': 109252775, 'iexRealtimePrice': 137.74, 'iexRealtimeSize': 37, 'iexLastUpdated': 1622055230526, 'delayedPrice': 135.05, 'delayedPriceTime': 1678542485777, 'oddLotDelayedPrice': 136.56, 'oddLotDelayedPriceTime': 1664882880451, 'extendedPrice': 135.38, 'extendedChange': -0.29, 'extendedChangePercent': -0.0023, 'extendedPriceTime': 1623539794187, 'previousClose': 136.59, 'previousVolume': 111150521, 'change': 1.14, 'changePercent': 0.00905,

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [29]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [30]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [31]:
final_dataframe = pd.DataFrame(columns= my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
            index = my_columns
        ),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,132.67,56.32,N/A
1,AAL,15.28,-1.09,N/A
2,AAP,172.80,25.34,N/A
3,AAPL,135.28,41.66,N/A
4,ABBV,108.76,23.44,N/A
...,...,...,...,...
500,YUM,107.41,33.33,N/A
501,ZBH,162.87,-218.78,N/A
502,ZBRA,406.45,47.41,N/A
503,ZION,50.91,20.84,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [32]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis = 1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,40.15,2.45,N/A
1,NLOK,22.06,4.06,N/A
2,AIV,5.08,4.84,N/A
3,UNM,24.08,5,N/A
4,BIO,624.38,5.39,N/A
5,KIM,15.77,7.1,N/A
6,ALL,113.33,8.1,N/A
7,MET,50.26,8.2,N/A
8,CPB,47.93,8.22,N/A
9,BIIB,259.89,8.49,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [33]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [34]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [35]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,40.15,2.45,498
1,NLOK,22.06,4.06,906
2,AIV,5.08,4.84,3937
3,UNM,24.08,5,830
4,BIO,624.38,5.39,32
5,KIM,15.77,7.1,1268
6,ALL,113.33,8.1,176
7,MET,50.26,8.2,397
8,CPB,47.93,8.22,417
9,BIIB,259.89,8.49,76


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [36]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()


#Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

#Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

#Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']


#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda


#Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit



Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [37]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
       enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
       ebitda = data[symbol]['advanced-stats']['EBITDA']
       gross_profit = data[symbol]['advanced-stats']['grossProfit']

       try:
            ev_to_ebitda = enterprise_value/ebitda
       except TypeError:
            ev_to_ebitda = np.NaN
       try:
            ev_to_gross_profit = enterprise_value/gross_profit
       except TypeError:
            ev_to_gross_profit = np.NaN


       rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            ],
            index = rv_columns),
                ignore_index = True
        )

In [38]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.26,N/A,56.32,N/A,8.32,N/A,7.3,N/A,34.018305,N/A,13.535703,N/A,N/A
1,AAL,15.64,N/A,-1.09,N/A,-1.76,N/A,0.3904,N/A,-5.973091,N/A,1.332018,N/A,N/A
2,AAP,171.28,N/A,25.73,N/A,3.06,N/A,1.16,N/A,11.623376,N/A,2.604537,N/A,N/A
3,AAPL,135.77,N/A,40.3,N/A,34.65,N/A,8.17,N/A,29.063708,N/A,21.544667,N/A,N/A
4,ABBV,111.55,N/A,23.75,N/A,12.53,N/A,4.73,N/A,15.666424,N/A,9.288884,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.34,N/A,32.57,N/A,-4.12,N/A,6,N/A,22.968449,N/A,10.056691,N/A,N/A
501,ZBH,166.02,N/A,-216.51,N/A,2.91,N/A,4.69,N/A,27.702286,N/A,7.935914,N/A,N/A
502,ZBRA,419.84,N/A,47.07,N/A,11.7,N/A,5.11,N/A,32.898001,N/A,12.112136,N/A,N/A
503,ZION,49.78,N/A,21.4,N/A,1.2,N/A,2.77,N/A,12.273323,N/A,3.134514,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [39]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,209.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,240.210,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.06,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,30.240,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.750,N/A,14.28,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,55.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.630,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,166.960,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1812.790,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.135,N/A,30.98,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [40]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [41]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [42]:
from scipy.stats import percentileofscore as score

metrics = {
            'Price-to-Earnings Ratio':'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio':'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.26,N/A,56.32,0.861386,8.32,0.746535,7.3000,0.805941,34.018305,0.845545,13.535703,0.794059,N/A
1,AAL,15.64,N/A,-1.09,0.144554,-1.76,0.0455446,0.3904,0.0217822,-5.973091,0.0277228,1.332018,0.0534653,N/A
2,AAP,171.28,N/A,25.73,0.479208,3.06,0.417822,1.1600,0.163366,11.623376,0.279208,2.604537,0.126733,N/A
3,AAPL,135.77,N/A,40.30,0.734653,34.65,0.962376,8.1700,0.841584,29.063708,0.811881,21.544667,0.942574,N/A
4,ABBV,111.55,N/A,23.75,0.443564,12.53,0.847525,4.7300,0.671287,15.666424,0.461386,9.288884,0.627723,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.34,N/A,32.57,0.623762,-4.12,0.0425743,6.0000,0.749505,22.968449,0.69901,10.056691,0.663366,N/A
501,ZBH,166.02,N/A,-216.51,0.019802,2.91,0.392079,4.6900,0.633663,27.702286,0.788119,7.935914,0.50297,N/A
502,ZBRA,419.84,N/A,47.07,0.778218,11.70,0.836634,5.1100,0.70099,32.898001,0.837624,12.112136,0.752475,N/A
503,ZION,49.78,N/A,21.40,0.410891,1.20,0.121782,2.7700,0.442574,12.273323,0.316832,3.134514,0.160396,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [43]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, ['RV Score']] = mean(value_percentiles)

rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.26,N/A,56.32,0.861386,8.32,0.746535,7.3000,0.805941,34.018305,0.845545,13.535703,0.794059,0.810693
1,AAL,15.64,N/A,-1.09,0.144554,-1.76,0.0455446,0.3904,0.0217822,-5.973091,0.0277228,1.332018,0.0534653,0.0586139
2,AAP,171.28,N/A,25.73,0.479208,3.06,0.417822,1.1600,0.163366,11.623376,0.279208,2.604537,0.126733,0.293267
3,AAPL,135.77,N/A,40.30,0.734653,34.65,0.962376,8.1700,0.841584,29.063708,0.811881,21.544667,0.942574,0.858614
4,ABBV,111.55,N/A,23.75,0.443564,12.53,0.847525,4.7300,0.671287,15.666424,0.461386,9.288884,0.627723,0.610297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,112.34,N/A,32.57,0.623762,-4.12,0.0425743,6.0000,0.749505,22.968449,0.69901,10.056691,0.663366,0.555644
501,ZBH,166.02,N/A,-216.51,0.019802,2.91,0.392079,4.6900,0.633663,27.702286,0.788119,7.935914,0.50297,0.467327
502,ZBRA,419.84,N/A,47.07,0.778218,11.70,0.836634,5.1100,0.70099,32.898001,0.837624,12.112136,0.752475,0.781188
503,ZION,49.78,N/A,21.40,0.410891,1.20,0.121782,2.7700,0.442574,12.273323,0.316832,3.134514,0.160396,0.290495


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [44]:
rv_dataframe.sort_values('RV Score', ascending= True, inplace= True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace= True)
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.41,N/A,5.00,0.158416,0.4467,0.049505,0.4182,0.0336634,2.590058,0.0336634,0.412087,0.0039604,0.0558416
1,AIG,40.91,N/A,-7.31,0.106931,0.5579,0.0554455,0.7741,0.0653465,4.683849,0.0455446,0.788875,0.0158416,0.0578218
2,AAL,15.64,N/A,-1.09,0.144554,-1.7600,0.0455446,0.3904,0.0217822,-5.973091,0.0277228,1.332018,0.0534653,0.0586139
3,HFC,25.68,N/A,-25.96,0.0673267,0.8037,0.0673267,0.3019,0.0138614,6.373862,0.0633663,2.283128,0.110891,0.0645545
4,MET,50.87,N/A,8.20,0.168317,0.6185,0.0613861,0.6886,0.0574257,4.664271,0.0415842,0.673180,0.0118812,0.0681188
5,F,9.00,N/A,-17.28,0.0811881,1.1700,0.113861,0.2834,0.00990099,4.679757,0.0435644,2.009494,0.0930693,0.0683168
6,BA,216.70,N/A,-27.50,0.0653465,-10.2500,0.0316832,2.0000,0.326733,-29.394071,0.0138614,-203.386486,0.0019802,0.0879208
7,HIG,52.00,N/A,10.82,0.19604,1.0800,0.0990099,0.8883,0.0891089,4.380789,0.039604,0.879714,0.019802,0.0887129
8,FTI,11.80,N/A,-0.95,0.146535,1.2200,0.129703,0.3952,0.0237624,3.506015,0.0356436,2.311746,0.114851,0.090099
9,ALL,113.42,N/A,7.80,0.164356,1.3700,0.170297,0.7849,0.0712871,2.573207,0.0316832,0.750227,0.0138614,0.090297


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [45]:
portfolio_input()

In [46]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.41,2135,5.00,0.158416,0.4467,0.049505,0.4182,0.0336634,2.590058,0.0336634,0.412087,0.0039604,0.0558416
1,AIG,40.91,1222,-7.31,0.106931,0.5579,0.0554455,0.7741,0.0653465,4.683849,0.0455446,0.788875,0.0158416,0.0578218
2,AAL,15.64,3196,-1.09,0.144554,-1.7600,0.0455446,0.3904,0.0217822,-5.973091,0.0277228,1.332018,0.0534653,0.0586139
3,HFC,25.68,1947,-25.96,0.0673267,0.8037,0.0673267,0.3019,0.0138614,6.373862,0.0633663,2.283128,0.110891,0.0645545
4,MET,50.87,982,8.20,0.168317,0.6185,0.0613861,0.6886,0.0574257,4.664271,0.0415842,0.673180,0.0118812,0.0681188
5,F,9.00,5555,-17.28,0.0811881,1.1700,0.113861,0.2834,0.00990099,4.679757,0.0435644,2.009494,0.0930693,0.0683168
6,BA,216.70,230,-27.50,0.0653465,-10.2500,0.0316832,2.0000,0.326733,-29.394071,0.0138614,-203.386486,0.0019802,0.0879208
7,HIG,52.00,961,10.82,0.19604,1.0800,0.0990099,0.8883,0.0891089,4.380789,0.039604,0.879714,0.019802,0.0887129
8,FTI,11.80,4237,-0.95,0.146535,1.2200,0.129703,0.3952,0.0237624,3.506015,0.0356436,2.311746,0.114851,0.090099
9,ALL,113.42,440,7.80,0.164356,1.3700,0.170297,0.7849,0.0712871,2.573207,0.0316832,0.750227,0.0138614,0.090297


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [57]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [58]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [59]:
column_formats = {
                    'A':['Ticker', string_template],
                    'B':['Price', dollar_template],
                    'C':['Number of Shares to Buy', integer_template],
                    'D':['Price-to-Earnings Ratio', float_template],
                    'E':['PE Percentile', percent_template],
                    'F':['Price-to-Book Ratio', float_template],
                    'G':['PB Percentile', percent_template],
                    'H':['Price-to-Sales Ratio', float_template],
                    'I':['PS Percentile', percent_template],
                    'J':['EV/EBITDA', float_template],
                    'K':['EV/EBITDA Percentile', percent_template],
                    'L':['EV/GP', float_template],
                    'M':['EV/GP Percentile', percent_template],
                    'N':['RV Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [60]:
writer.save()